In [1]:
import torch
import transformers
import json
import os
print("Torch version:{}".format(torch.__version__)) # Torch version:1.12.1
print("Cuda version: {}".format(torch.version.cuda)) # Cuda version: 11.3
print("transformers version: {}".format(transformers.__version__)) # transformers 4.28.0
print("GPU 사용 가능여부: {}".format(torch.cuda.is_available()))

Torch version:1.12.1
Cuda version: 11.3
transformers version: 4.28.0
GPU 사용 가능여부: True


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 데이터 준비

In [3]:
# data_path_1_SFT = '/aiffel/aiffel/GD9/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl' 
# with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
#     list_data_dict = json.load(json_file)

# print(len(list_data_dict))
# list_data_dict[:3]

In [4]:
# base_path = "/aiffel/aiffel/GD9/dataset/train"
# file_prefix = "korquad2.1_train_"
# file_count = 39  # 00부터 38까지

# for i in range(file_count):
#     file_name = f"{file_prefix}{str(i).zfill(2)}.json"
#     file_path = os.path.join(base_path, file_name)

#     try:
#         with open(file_path, "r", encoding="utf-8") as json_file:
#             json.load(json_file)
#         print(f"파일 정상 확인: {file_name}")
#     except Exception as e:
#         print(f"JSON 파싱 에러 발생: {file_name} - {e}")

In [5]:
# # 새로운 데이터 경로 및 파일 정보
# base_path = "/aiffel/aiffel/GD9/dataset/train"
# file_prefix = "korquad2.1_train_"
# file_count = 39  # 00부터 38까지

# # 새로운 데이터를 변환하여 기존 데이터에 추가
# try:
#     for i in range(file_count):
#         file_name = f"{file_prefix}{str(i).zfill(2)}.json"
#         file_path = os.path.join(base_path, file_name)

#         # 파일 열기 및 변환
#         with open(file_path, "r", encoding="utf-8") as json_file:
#             data = json.load(json_file)

#         # 'data' 키의 내용 처리
#         if "data" in data:
#             for entry in data["data"]:
#                 for qas_entry in entry.get("qas", []):
#                     question = qas_entry.get("question", "")
#                     answer = qas_entry.get("answer", {}).get("text", "")

#                     # prompt와 completion 생성
#                     prompt = question
#                     completion = answer

#                     # 토크나이저를 이용해 토큰 길이 계산
#                     tokens = len(tokenizer.encode(completion, add_special_tokens=False))

#                     # 저장 형식에 맞게 추가
#                     list_data_dict.append({
#                         "prompt": prompt,
#                         "completion": completion,
#                         "tokens": tokens
#                     })

#     # 최종 결과 확인
#     print(f"합쳐진 데이터 개수: {len(list_data_dict)}")
#     print("첫 번째 데이터:")
#     print(json.dumps(list_data_dict[:1], ensure_ascii=False, indent=2))

# except Exception as e:
#     print(f"새 데이터 처리 에러: {e}")

# # 결과를 새 파일로 저장
# output_path = "/aiffel/aiffel/GD9/KoChatGPT/data_kochatgpt/merged_data.jsonl"
# try:
#     with open(output_path, "w", encoding="utf-8") as output_file:
#         json.dump(list_data_dict, output_file, ensure_ascii=False, indent=2)
#     print(f"결과가 저장되었습니다: {output_path}")
# except Exception as e:
#     print(f"저장 에러: {e}")

In [6]:
# # JSONL 파일 경로
# merged_data_path = "/aiffel/aiffel/GD9/KoChatGPT/data_kochatgpt/merged_data.jsonl"

# # JSONL 파일 읽어서 list_data_dict로 저장
# try:
#     with open(merged_data_path, "r", encoding="utf-8") as json_file:
#         list_data_dict = json.load(json_file)  # JSONL 파일 읽기

#     # 데이터 확인
#     print(f"데이터 개수: {len(list_data_dict)}")
#     print("첫 번째 데이터:")
#     print(json.dumps(list_data_dict[:20], ensure_ascii=False, indent=2))
# except FileNotFoundError:
#     print(f"파일을 찾을 수 없습니다: {merged_data_path}")
# except json.JSONDecodeError as e:
#     print(f"JSON 디코딩 오류 발생: {e}")
# except Exception as e:
#     print(f"에러 발생: {e}")

In [7]:
# len(list_data_dict)

In [8]:
# import matplotlib.pyplot as plt

# # 데이터에서 tokens 추출
# tokens = [entry['tokens'] for entry in list_data_dict]

# # 그래프 그리기
# plt.figure(figsize=(8, 4))
# plt.plot(tokens, marker='o')  # 선 그래프
# plt.xlabel('Index')
# plt.ylabel('Tokens')
# plt.title('Tokens per Index')
# plt.grid()
# plt.tight_layout()
# plt.show()

In [9]:
# # tokens >= 4000
# large_completions = [entry['completion'] for entry in list_data_dict if entry['tokens'] >= 3000]

# # 결과 출력
# large_completions

In [10]:
import re

def preprocess_data(data):
    """
    데이터를 정규표현식을 사용해 전처리하는 함수.

    Parameters:
        data (list): 전처리할 데이터 리스트.

    Returns:
        list: 전처리된 데이터를 담은 리스트.
    """
    processed_data = []

    for entry in data:
        # HTML 태그 제거
        cleaned_prompt = re.sub(r'<.*?>', '', entry['prompt'])
        cleaned_completion = re.sub(r'<.*?>', '', entry['completion'])

        # ( ) 괄호와 괄호 안 내용 제거
        cleaned_prompt = re.sub(r'\(.*?\)', '', cleaned_prompt)
        cleaned_completion = re.sub(r'\(.*?\)', '', cleaned_completion)

        # 한글, 영어, 숫자, !?.'" 외의 문자 제거
        cleaned_prompt = re.sub(r'[^가-힣a-zA-Z0-9!?.]', ' ', cleaned_prompt)
        cleaned_completion = re.sub(r'[^가-힣a-zA-Z0-9!?.]', ' ', cleaned_completion)

        # !?.'" 중복 제거
        cleaned_prompt = re.sub(r'(\!|\?|\.|\'|\"){2,}', lambda x: x.group(0)[0], cleaned_prompt)
        cleaned_completion = re.sub(r'(\!|\?|\.|\'|\"){2,}', lambda x: x.group(0)[0], cleaned_completion)
        
        # "token 숫자" 패턴 제거 (숫자가 1~3자리, 주변 공백 포함)
        cleaned_prompt = re.sub(r'\btoken\s*\d{1,3}\b', '', cleaned_prompt)
        cleaned_completion = re.sub(r'\btoken\s*\d{1,3}\b', '', cleaned_completion)

        # 연속된 공백 하나로 통일
        cleaned_prompt = re.sub(r'\s+', ' ', cleaned_prompt).strip()
        cleaned_completion = re.sub(r'\s+', ' ', cleaned_completion).strip()

        # 토크나이저를 사용해 tokens 계산
        prompt_tokens = len(tokenizer.encode(cleaned_prompt, add_special_tokens=False))
        completion_tokens = len(tokenizer.encode(cleaned_completion, add_special_tokens=False))

        processed_data.append({
            'prompt': cleaned_prompt,
            'completion': cleaned_completion,
            'tokens': completion_tokens
        })

    return processed_data

In [11]:
# list_data_dict = preprocess_data(list_data_dict)
# list_data_dict

In [12]:
# # 전처리된 데이터에서 tokens 추출
# tokens = [entry['tokens'] for entry in list_data_dict]

# # 그래프 그리기
# plt.figure(figsize=(8, 4))
# plt.plot(tokens, marker='o')  # 선 그래프
# plt.xlabel('Index')
# plt.ylabel('Tokens')
# plt.title('Tokens per Index')
# plt.grid()
# plt.tight_layout()
# plt.show()

In [13]:
# # tokens > 512
# large_completions = [entry['completion'] for entry in list_data_dict if entry['tokens'] > 512]

# # 결과 출력
# large_completions

In [14]:
# len(large_completions)

In [15]:
# # list_data_dict의 prompt를 토큰화하여 토큰 수 계산
# token_counts = [len(tokenizer.encode(entry['prompt'], add_special_tokens=False)) for entry in list_data_dict]

# # 그래프 그리기
# plt.figure(figsize=(12, 6))
# plt.plot(token_counts, marker='o', linestyle='-', label='Token Counts')
# plt.xlabel('Index')
# plt.ylabel('Token Count')
# plt.title('Token Counts for Prompts')
# plt.grid()
# plt.legend()
# plt.tight_layout()
# plt.show()

In [16]:
# # tokens > 80 필터링
# large_prompts = [list_data_dict[i]['prompt'] for i, count in enumerate(token_counts) if count > 80]

# # 결과 출력
# large_prompts

In [17]:
# len(large_prompts)

In [18]:
# # 조건에 따라 데이터 필터링
# filtered_data = []
# for entry in list_data_dict:
#     # "completion" 단어가 prompt에 포함되어 있으면 제거
#     if "completion" in entry['prompt']:
#         continue

#     # completion의 토큰 수 계산
#     completion_tokens = len(tokenizer.encode(entry['completion'], add_special_tokens=False))

#     # completion의 토큰 수가 512를 초과하면 제거
#     if completion_tokens > 512:
#         continue

#     # 조건을 만족하면 데이터 추가
#     filtered_data.append(entry)

# # drop
# list_data_dict = filtered_data

# len(list_data_dict)

In [19]:
# list_data_dict

In [20]:
# # 데이터 저장 경로
# output_path = "/aiffel/aiffel/GD9/dataset/cleaned_data.json"

# # JSON 파일로 저장
# with open(output_path, 'w', encoding='utf-8') as f:
#     json.dump(list_data_dict, f, ensure_ascii=False, indent=4)

# # 결과 출력
# print(f"Filtered data count: {len(list_data_dict)}")
# print(f"Data saved to {output_path}")

In [21]:
import json

# 파일 경로 설정
file_path = "/aiffel/aiffel/GD9/dataset/cleaned_data.json"

# JSON 파일 읽기
try:
    with open(file_path, "r", encoding="utf-8") as f:
        list_data_dict = json.load(f)
    print(f"Data successfully loaded. Total entries: {len(list_data_dict)}")
except FileNotFoundError:
    print(f"File not found at {file_path}")
except json.JSONDecodeError:
    print(f"Error decoding JSON at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Data successfully loaded. Total entries: 131216


In [22]:
# prompt에 텍스트 추가
for entry in list_data_dict:
    if 'prompt' in entry:
        entry['prompt'] = f"다음 질문에 단계별로 설명해라. {entry['prompt']}"

# 결과 확인
print("Updated data examples:")
for i, entry in enumerate(list_data_dict[:5]):  # 상위 5개 예시 출력
    print(f"{i+1}: {entry['prompt']}")

Updated data examples:
1: 다음 질문에 단계별로 설명해라. 불고기용 고기 한우에요?
2: 다음 질문에 단계별로 설명해라. 쓰던 앱이 유료로 전환됐어
3: 다음 질문에 단계별로 설명해라. 여친이랑 다툼
4: 다음 질문에 단계별로 설명해라. 술 먹고 싶어
5: 다음 질문에 단계별로 설명해라. 잊고싶다.


In [23]:
len(list_data_dict)

131216

In [24]:
import random
random_seed = 42
random.seed(random_seed)

# 랜덤 추출 비율
sample_ratio = 0.4

# 랜덤으로 40% 데이터 추출
sample_size = int(len(list_data_dict) * sample_ratio)  # 40% 크기 계산
list_data_dict = random.sample(list_data_dict, sample_size)

# 결과 확인

print(f"추출된 데이터 개수: {len(list_data_dict)}")

추출된 데이터 개수: 52486


In [25]:
# # 프롬프트 엔지니어링
# for entry in list_data_dict:
#     entry['prompt'] = f"다음 질문에 반드시 답변해라. {entry['prompt']}"

# # 변경된 데이터 확인 (샘플 출력)
# print("Sample engineered data:")
# print(list_data_dict[0])

In [26]:
# # train과 val 데이터 나누기 (80%:20%)
from sklearn.model_selection import train_test_split
# train과 validation 데이터 분리
train_data, val_data = train_test_split(list_data_dict, test_size=0.2, random_state=42)

In [27]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

In [28]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

print(tokenizer)

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True)


In [29]:
from typing import Optional, Dict, Sequence
class SFT_dataset(Dataset):
    def __init__(self, data: list, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in data:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in data:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))

    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [30]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [31]:
train_dataset = SFT_dataset(data=train_data, tokenizer=tokenizer)
val_dataset = SFT_dataset(data=val_data, tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7182,  8139, 45211, 12314, 11533, 44373, 25462,
        30613, 35423, 10203, 16446, 20929,  9341,   375,   375,   378,   378,
          378, 41951,   454,  9549, 20549,   383,  8142,  7192, 14914,  8263,
        12744, 46201, 22816, 40983,  9022,  9044, 14150, 10615,  9144, 16691,
        14415, 14415,  9528, 14373, 23881, 11412,  9534, 30705, 35423, 24313,
        12521, 14415, 10153, 25420, 11716, 11347, 10113,  9080, 26715, 19392,
        28469, 24313, 12521, 14415,  9522, 11716, 23881, 26681,  9080, 25565,
        14125, 13213,  6903, 25089,  9098, 11942, 26757,  9600, 26083, 24313,
        12521, 14415, 19882, 17632, 23881, 10732, 43414,  9080, 12775, 13488,
        19208,  6903, 24313, 12521, 14415, 20436, 20436, 11347,  8135, 13996,
        20436, 28469, 24313, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
    

In [32]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="/aiffel/aiffel/GD9/KoChatGPT/test",
    num_train_epochs=1,  # 한 번에 1 epoch만 수행
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True,
    overwrite_output_dir=True
)

total_epochs = 2

for epoch in range(total_epochs):
    print(f"Epoch {epoch+1}/{total_epochs}")
    
    # Trainer 생성
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )
    
    # 학습
    trainer.train()
    
    # 모델 저장
    model.save_pretrained(f"/aiffel/aiffel/GD9/KoChatGPT/output_1_SFT/least_to_most_epoch_{epoch+1}")
    tokenizer.save_pretrained(f"/aiffel/aiffel/GD9/KoChatGPT/output_1_SFT/least_to_most_epoch_{epoch+1}")

    # 저장된 모델 로드
    model = AutoModelForCausalLM.from_pretrained(f"/aiffel/aiffel/GD9/KoChatGPT/output_1_SFT/least_to_most_epoch_{epoch+1}").to(device)

Epoch 1/2


/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.838400,3.710028


Epoch 2/2


/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.693100,3.722422


In [33]:
# Prompt 생성
PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}
# 샘플 Prompt
sample_prompt = "리처드 닉슨이 43대 부통령직을 수행한 년도는?"
input_ids = tokenizer(PROMPT_DICT['prompt_input'].format_map({'prompt': sample_prompt}), return_tensors="pt").input_ids.to(device)

# 텍스트 생성
output = model.generate(input_ids, max_length=64, num_beams=4, no_repeat_ngram_size=4)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated Text:")
print(generated_text)

Generated Text:
### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):1994년


In [34]:
print(tokenizer.eos_token_id)

1


### test data

In [51]:
# 테스트 데이터 불러오기
test_base_path = "/aiffel/aiffel/GD9/dataset/test"
test_files = [f"korquad2.1_dev_{str(i).zfill(2)}.json" for i in range(5)]

test_data = []
for test_file in test_files:
    file_path = os.path.join(test_base_path, test_file)
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    if "data" in data:
        for entry in data["data"]:
            for qas_entry in entry.get("qas", []):
                question = qas_entry.get("question", "")
                answer = qas_entry.get("answer", {}).get("text", "")
                # 키 변경
                test_data.append({"prompt": question, "completion": answer})

# 결과 확인
print(f"Test 데이터 개수: {len(test_data)}")
print(f"첫 번째 데이터: {test_data[0]}")

Test 데이터 개수: 10165
첫 번째 데이터: {'prompt': '파스칼레 소틸레의 스파이크 높이는 몇 cm인가?', 'completion': '<td>332cm</td>'}


In [52]:
test_data

[{'prompt': '파스칼레 소틸레의 스파이크 높이는 몇 cm인가?', 'completion': '<td>332cm</td>'},
 {'prompt': '2016년 하계 올림픽 캐나다 남자 배구 대표팀 선수들 중, 타일러 샌더스는 2015-16년에 어느 팀 소속이었나?',
  'completion': 'MKS 벵진'},
 {'prompt': '혈액 검사에서 나트륨의 정상범위는 최소 136mmol/L에서 최대 몇 mmol/L까지 일까?',
  'completion': '<td>145</td>'},
 {'prompt': '안토닌 드보르자크의 작품 중 18개의 가곡으로 이루어진 노래는?', 'completion': '사이프러스'},
 {'prompt': '에드먼드 핼리가 달의 운동이 빨라지고 있다고 주장한 년도는 언제일까?', 'completion': '1695년'},
 {'prompt': '지구의 속도를 낮추고 달의 궤도를 올라가게 하는 것은 무엇일까?', 'completion': '중력 토크'},
 {'prompt': '조영무의 가족관계는?',
  'completion': '<ul><li>할아버지\xa0: 조순후, 증 참찬문하부사</li>\n<li>할머니\xa0: 증 정부인 문천박씨(文川朴氏)</li>\n<li>아버지\xa0: <a>조세진</a></li>\n<li>어머니\xa0: 증 정경부인 영흥최씨(贈貞敬夫人永興崔氏)</li>\n<li>부인\xa0: 신천김씨\n<ul><li>아들\xa0: <a>조서</a>(趙敍, <a>1370년</a> ~ <a>1429년</a>)</li>\n<li>아들\xa0: 조윤(趙倫)</li>\n<li>아들\xa0: 조전(趙琠)</li></ul></li>\n<li>부인\xa0: 재령강씨(載寧康氏)\n<ul><li>아들\xa0: 조질(趙秩)</li>\n<li>아들\xa0: 조이(趙理)</li></ul></li></ul>'},
 {'prompt': '조선시대 조영무와  왕자의 난에는 어떤 관계가 있을까?',
  'completion': 

In [53]:
test_data = preprocess_data(test_data)

In [54]:
# prompt에 텍스트 추가
for entry in test_data:
    if 'prompt' in entry:
        entry['prompt'] = f"다음 질문에 단계별로 설명해라. {entry['prompt']}"
# 다음 질문에 여러관점으로 설명해라.
# 결과 확인
print("Updated data examples:")
for i, entry in enumerate(test_data[:5]):  # 상위 5개 예시 출력
    print(f"{i+1}: {entry['prompt']}")

Updated data examples:
1: 다음 질문에 단계별로 설명해라. 파스칼레 소틸레의 스파이크 높이는 몇 cm인가?
2: 다음 질문에 단계별로 설명해라. 2016년 하계 올림픽 캐나다 남자 배구 대표팀 선수들 중 타일러 샌더스는 2015 16년에 어느 팀 소속이었나?
3: 다음 질문에 단계별로 설명해라. 혈액 검사에서 나트륨의 정상범위는 최소 136mmol L에서 최대 몇 mmol L까지 일까?
4: 다음 질문에 단계별로 설명해라. 안토닌 드보르자크의 작품 중 18개의 가곡으로 이루어진 노래는?
5: 다음 질문에 단계별로 설명해라. 에드먼드 핼리가 달의 운동이 빨라지고 있다고 주장한 년도는 언제일까?


In [55]:
# 랜덤 시드 고정
random_seed = 42
random.seed(random_seed)

# 40% 샘플링
sample_ratio = 0.4
sample_size = int(len(test_data) * sample_ratio)

# 랜덤 샘플링
test_data = random.sample(test_data, sample_size)

#### BERT_SCORE

In [40]:
# pip install bert-score

In [41]:
from bert_score import score  
from tqdm import tqdm

In [56]:
generator = pipeline('text-generation', model='/aiffel/aiffel/GD9/KoChatGPT/output_1_SFT/least_to_most_epoch_1', tokenizer=tokenizer,device=0)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=tokenizer.eos_token_id,  # 올바른 종료 토큰 ID 사용
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

# Prompt 생성
PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}
list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': entry['prompt']}) for entry in test_data]

# Completion 생성 
batch_size = 64
# "### Response(응답):" 이후의 텍스트만 추출
generated_completions = []
for i in tqdm(range(0, len(list_prompt), batch_size), desc="Generating completions"):
    batch_prompts = list_prompt[i:i + batch_size]
    batch_results = generator(batch_prompts, **generation_args)
    for res in batch_results:
        text = res[0]['generated_text']
        # "### Response(응답):" 이후 텍스트 추출
        response = text.split("### Response(응답):")[1].strip() if "### Response(응답):" in text else text
        generated_completions.append(response)

Generating completions:   0%|          | 0/64 [00:00<?, ?it/s]/opt/conda/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Generating completions:  16%|█▌        | 10/64 [02:31<13:07, 14.58s/it]/opt/conda/lib/python3.9/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Generating completions: 100%|██████████| 64/64 [15:23<00:00, 14.43s/it]


In [57]:
# BERTScore 계산
original_completions = [entry['completion'] for entry in test_data]

# BERTScore 계산
P, R, F1 = score(
    generated_completions,
    original_completions,
    lang="ko",  # 한국어 사용
    verbose=True
)

# 결과 출력
print(f"BERTScore Precision: {P.mean().item():.4f}")
print(f"BERTScore Recall: {R.mean().item():.4f}")
print(f"BERTScore F1: {F1.mean().item():.4f}")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/98 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/64 [00:00<?, ?it/s]

done in 21.96 seconds, 185.14 sentences/sec
BERTScore Precision: 0.7523
BERTScore Recall: 0.7261
BERTScore F1: 0.7375


#### ROUGE

In [44]:
# pip install rouge_score

In [60]:
from datasets import load_metric

# ROUGE 메트릭 로드
rouge = load_metric("rouge")

# 생성된 텍스트와 정답 텍스트 준비
generated_texts = generated_completions  # generator로 생성된 텍스트
reference_texts = [entry['completion'] for entry in test_data]  # 정답 텍스트

# ROUGE 평가
results = rouge.compute(predictions=generated_texts, references=reference_texts)

# 결과 출력
print("ROUGE Evaluation Results:")
for key, value in results.items():
    print(f"{key}: {value.mid}")

ROUGE Evaluation Results:
rouge1: Score(precision=0.04289392104043863, recall=0.023176079114535612, fmeasure=0.02486726083579298)
rouge2: Score(precision=0.0032781007642552432, recall=0.001156204457883472, fmeasure=0.0013640390484624758)
rougeL: Score(precision=0.039799949665076426, recall=0.02155677322176967, fmeasure=0.023185260843066775)
rougeLsum: Score(precision=0.03985264831098122, recall=0.02171140309350875, fmeasure=0.02324907286846604)


#### 정성평가

In [46]:
# 출력: Prompt, Generated Completion, Original Completion
for i in range(len(test_data)):
    print(f"### Question {i+1}:")  # 질문 번호
    print(f"Prompt: {test_data[i]['prompt']}")
    print()
    print(f"### Generated Completion:")
    print(generated_completions[i])
    print()
    print(f"### Original Completion:")
    print(test_data[i]['completion'])
    print("="*80)  # 구분선

### Question 1:
Prompt: 브리트니 스피어스의 공개되지 않은 노래와 공개된 데모버전 목록은 어떻게 되는가?

### Generated Completion:
앨범 정보 트랙 리스트 1 Let Me Sweet Up 발매일 2016년 1월 1일

### Original Completion:
1999 Luv the Hurt Away 2003 Get It 2006 Little Me 36 2006 Rebellion 2007 Cold as Fire 2007 Baby Boy 37 2008 All That She Wants
### Question 2:
Prompt: 당시 신라로부터 압박받던 나카노오에의 전략은 뭐였어?

### Generated Completion:
나카노오로의 입김

### Original Completion:
국토 방위를 중심으로 국내 제도 정비
### Question 3:
Prompt: 차르키다드리 상공에서 공중 충돌한 두 항공기를 관제하던 관제사는 누구야?

### Generated Completion:
아우구스트 모나코

### Original Completion:
VK 두타
### Question 4:
Prompt: 공감토크쇼 놀러와가 2008년 2012년까지 방영된 요일은?

### Generated Completion:
금요일

### Original Completion:
월요일
### Question 5:
Prompt: 설형문자를 해석하기 위해 어떤 노력이 있었나요?

### Generated Completion:
설형문자는 고대 그리스 문자를 의미한다.

### Original Completion:
설형문자를 해독하는데 결정적인 단서를 제공한 것은 베히스툰 비문이다. 베히스툰 비문은 고대 페르시아어 엘람어 아카드어의 세 언어로 같은 내용을 새겨놓았다. 1802년 독일의 금석학자인 게오르그 프리드리히 그로테펜트가 설형문자의 해독을 시작하였다. 이후 1838년 영국의 핸리 로우린슨이 그로테펜트의 작업 내용과 현대 페르시아어에 대한

### Original Completion:
2006
### Question 255:
Prompt: 같은 볼륨 안에서 파일을 이동시킬때 어떤점을 주의해야할까요?

### Generated Completion:
파일의 크기가 작은 경우 파일의 크기는 작아진다. 따라서 파일의 크기나 용량을 선택해야 한다.

### Original Completion:
단순히 원본 파일을 복사하거나 삭제하는 것이 아니라는 점을 주의해야 한다.
### Question 256:
Prompt: AAB 패턴이란 무엇입니까?

### Generated Completion:
ABS

### Original Completion:
AAB 패턴이라 칭하는데 음악의 4마디를 먼저 구성하고 그 다음 4마디는 앞 4마디를 반복하며 다음 4마디는 끝나는 구절로 길게 만들었다
### Question 257:
Prompt: 유희왕 듀얼 링크스에 등장하는 NPC 듀얼리스트들의 특징은 무엇입니까?

### Generated Completion:
듀얼링크스의 멤버 중 한 명인 제이크가 그 주인공이다.

### Original Completion:
듀얼 월드에 상시로 나타나며 사용하는 덱은 NPC 덱 중 랜덤으로 구성된다. NPC 듀얼리스트로 플레이할 수는 없다. 일부 캐릭터는 한정된 듀얼 월드에서만 등장한다. 활기찬 학생 듀얼 월드 DM에서만 등장 쿨한 학생 듀얼 월드 DM에서만 등장 과묵한 학생 듀얼 월드 DM에서만 등장 절호조의 학생 듀얼 월드 DM에서만 등장 듀얼을 좋아하는 학생 듀얼 월드 DM에서만 등장 마이페이스인 학생 듀얼 월드 DM에서만 등장 에반 듀얼 월드 GX에서만 등장 로건 듀얼 월드 GX에서만 등장 재커리 듀얼 월드 GX에서만 등장 카일리 듀얼 월드 GX에서만 등장 알리사 듀얼 월드 GX에서만 등장 매디슨 듀얼 월드 GX에서만 등장 활기찬 소년 어른스러운 소년 깜찍한 소녀 심약한 소녀 시원한 청년 예의바른 청년 밝은 아가씨 친절한 아가씨 이름 없는 듀얼리스트 유희왕 태그 포스의 주인공으로 듀얼


### Generated Completion:
선거 전략

### Original Completion:
보트빌더
### Question 295:
Prompt: 1936년 하계 올림픽에서 독일 축구 국가대표팀은 몇 위를 차지했을까?

### Generated Completion:
2위

### Original Completion:
5위
### Question 296:
Prompt: 나홍선이 태어난 년도는 언제야?

### Generated Completion:
1914년

### Original Completion:
1504
### Question 297:
Prompt: 법원의 서열 및 상호저촉 등을 관장하는 법은 무엇인가?

### Generated Completion:
민법

### Original Completion:
입법법
### Question 298:
Prompt: 19세기 후반의 이탈리아 문학 중 세기말 문학을 소위 일컬어 무엇이라고 부르는가?

### Generated Completion:
기하학

### Original Completion:
데카당스 문학
### Question 299:
Prompt: 뇌파 영역 중 어느 영역에서 강한 활성이나타났는가?

### Generated Completion:
신경계

### Original Completion:
감마파
### Question 300:
Prompt: 3선 개헌 소식 때 이승만 대통령에게 조언을 했던 김영삼의 나이는?

### Generated Completion:
1960년

### Original Completion:
28세
### Question 301:
Prompt: 제 19대 총선에서 김윤덕은 무슨 선거구에 출마했을까?

### Generated Completion:
서울 중구

### Original Completion:
전북 전주시 완산구 갑
### Question 302:
Prompt: 벽운의 안쪽을 뭐라고 할까?

### Generated Completion:
사운드

###

### Question 530:
Prompt: 불법선거자금 모금을 한것이 밝혀져 이러난 결과는?

### Generated Completion:
죄송합니다 저는 AI 어시스턴트이기 때문에 정확한 답변을 제공할 수 없습니다. 죄송합니다.

### Original Completion:
기소
### Question 531:
Prompt: 10세기의 캔터베리 대주교에는 누가 있었을까?

### Generated Completion:
요한 바오로 2세

### Original Completion:
재임 기간 주교명 비고 914년 923년 애셀름Athelm 923년 941년 울프헬름Wulfhelm 942년 958년 오도Odo 959년 959년 앨프사이지Alfsige 959년 959년 비어셀름Birthelm 960년 988년 던스탄Dunstan 988년 989년 애셀가르Athelgar 990년 994년 시게릭Sigeric 995년 1005년 앨프릭Aelfric
### Question 532:
Prompt: 페가수스자리 IK A가 태양계를 향한 방향에서의 최대 속도는 얼마인가?

### Generated Completion:
2.5

### Original Completion:
41.5 km s
### Question 533:
Prompt: 1967년 당시 선진 기술을 전수받기 위해 정부와 기업이 초청한 외국인들의 거주시설로 지어진 아파트의 이름은 무엇인가?

### Generated Completion:
삼성아파트

### Original Completion:
힐탑아파트
### Question 534:
Prompt: 조지 암스트롱 커스터가 미육군 사관학교에 들어가기 전 교사를 지낸 주는 어디인가요?

### Generated Completion:
미 육군사관학교

### Original Completion:
오하이오
### Question 535:
Prompt: 픽업이 유일하게 감지할 수 있는 진동은?

### Generated Completion:
전류

### Orig

### Original Completion:
그녀의 남편인 모리스 마이어슨과 함께 1921년에 팔레스타인으로 이주하였고 키부츠에 가입하였다. 그녀의 일은 아몬드를 따고 닭을 돌보고 부엌에서 일하는 것이었다. 시간이 지나면서 그녀의 남편은 키부츠에서의 생활에 싫증이 나서 1924년에 떠났다. 예루살렘으로 가기 전에 잠시 텔 아비브에 머물렀다. 점점 둘은 멀어졌으나 이혼하지는 않았다. 그녀의 남편은 1951년 숨졌다. 1949년에서 1956년까지 노동 총리를 지냈고 1956년 데이비드 벤구리온 아래에서 공사가 되었다. 벤구리온이 그녀의 이름을 히브리어 이름으로 바꾸라고 요구하자 그녀의 성을 메이어 로 바꾸었다. 1960년대 초 그녀는 림프종이 있다는 것을 발견하였으나 1965년에 병이 심하게 악화될 때까지 비밀로 숨겼다. 총리 레비 에슈콜이 돌연사하자 1969년에 연합당은 그녀가 그의 자리를 잇도록 선택하였다.
### Question 801:
Prompt: 시간에 따른 데이터의 변화를 나타낸 직선을 무엇이라고 할까?

### Generated Completion:
데이터

### Original Completion:
추세선
### Question 802:
Prompt: 파울로 완초페의 선수 경력은 어떻게 될까?

### Generated Completion:
완초페는 2006년 FIFA 월드컵에 출전하여 UEFA 챔피언스리그에서 첫 경기를 치렀다. 이 경기에서 완초페는 2 0으로 비겼으며 팀은 준결승까지 진출했다. 2007년 FIFA 컨페더레이션스컵에서는 조별 예선 탈락과 함께 결승에 진출하지

### Original Completion:
완초페는 1997년 코스타리카의 에레디아노 에서 잉글랜드 프리미어리그의 더비 카운티로 이적하였다. 그는 데뷔전인 맨체스터 유나이티드와 경기에서 골을 기록하는 등 인상적인 활약을 하였고 1999년 웨스트 햄 유나이티드로 이적하였다. 그 후 맨체스터 시티에서도 활약하였으나 2004년 50만 파운드의 이적료로 스페인 프리메라리가의 말

...

중형차

### Original Completion:
전륜구동 방식
### Question 3450:
Prompt: 보아가 대한민국에서 발매한 음반에는 어떤 음반들이 있을까?

### Generated Completion:
앨범 정보 트랙 리스트 1 Let Me Down 2 Summer Girl Angel 3

### Original Completion:
순서 앨범 정보 활동곡 판매량 1 ID Peace B 발매일 2000년 8월 25일 보아의 데뷔 앨범. ID Peace B SARA 177 612장 월간 최고순위 16위 한국음반산업협회 2000년 판매량 59위 2 No.1 발매일 2002년 4월 13일 NO.1 My Sweetie 늘. gt 560 326장 월간 최고순위 1위 한국음반산업협회 2002년 음반판매량 4위 3 Atlantis Princess 발매일 2003년 5월 30일 아틀란티스 소녀 Milky Way 345 313장 월간 최고순위 1위 한국음반산업협회 2003년 판매량 5위 4 My Name 발매일 2004년 6월 11일 My Name Spark My Prayer 201 699장 월간 최고순위 1위 한국음반산업협회 2004년 판매량 11위 5 Girls On Top 발매일 2005년 6월 24일 MOTO 발매일 2005년 8월 17일 Girls On Top 의 리패키지 앨범. Girls On Top MOTO 117 416장 월간 최고순위 3위 한국음반산업협회 2005년 판매량 14위 한국음반산업협회 2006년 상반기 판매량 74위 6 Hurricane Venus 발매일 2010년 8월 5일 Copy amp Paste 발매일 2010년 9월 27일 Hurricane Venus 의 리패키지 앨범. Hurricane Venus Copy amp Paste 68 000장 월간 최고순위 1위 한터차트 2010년 판매량 5위 가온차트 2010년 판매량 22위 7 Only One 발매일 2012년 7월 25일 Only One The Shadow 34 255

### Question 3468:
Prompt: 닝보시의 1971 2000년도 중 1월 평균 상대 습도는 어떻게 되나요?

### Generated Completion:
76

### Original Completion:
76
### Question 3469:
Prompt: 2005년 유엔 인권 위원회 참가국 현황은 어떻게 되는가?

### Generated Completion:
유엔 인권 위원회는 세계인권선언의 이행과 국제사회에서의 인권 증진 등을 위한 국제협력 및 협력 활동을 수행하고 있다. 유엔 인권 위원회는 전 세계 인권 증진을 위한 국제협력의 중요한 역할을 수행하고 있다. 유엔 인권이사회는 매년 총회에서 다양한 인권 이슈를 논의하고 있으며 특히 유엔 인권이사회의 주요 활동 중 하나이다. 유엔

### Original Completion:
아프리카 15개국 부르키나파소 콩고 공화국 이집트 에리트레아 에티오피아 가봉 기니 케냐 모리타니 나이지리아 남아프리카 공화국 수단 스와질란드 토고 짐바브웨 아시아 12개국 부탄 중화인민공화국 인도 인도네시아 일본 말레이시아 네팔 파키스탄 카타르 대한민국 사우디아라비아 스리랑카 동유럽 5개국 아르메니아 헝가리 루마니아 러시아 우크라이나 라틴 아메리카 및 카리브 11개국 아르헨티나 브라질 코스타리카 쿠바 도미니카 공화국 에콰도르 과테말라 온두라스 멕시코 파라과이 페루 서유럽 및 기타 10개국 오스트레일리아 캐나다 핀란드 프랑스 독일 아일랜드 이탈리아 네덜란드 영국 미국
### Question 3470:
Prompt: 2016하계 올림픽 배구 이탈리아 멘네가티팀의 세트 득실은 어떻게되는가?

### Generated Completion:
2

### Original Completion:
1.000
### Question 3471:
Prompt: 개화파의 반대 세력을 뭐라고 하나?

### Generated Completion:
보수파

### Original Completion:
척사위정파
### Question 3472:

1920년대 초반 코펜하겐 덴마크에서 살고 있는 일러스트레이터 겸 예술가 게르다 베게너 는 그녀의 남편 에이나르 베게너 에게 여장을 하고 자신의 초상화에 여성 모델로 서달라는 요청한다. 초상화의 인기에 더 나아가 게르다는 그녀의 남편을 모델로 더 많은 그림을 그리게 된다. 에이나르는 이를 계기로 여성의 외모에 대한 매력을 느끼게 되고 릴리 엘베 라는 이름의 여자로서 또 다른 삶을 시작한다. 결국 엘베는 자신의 성 정체성에 눈을 뜨게 되고 세계 최초로 성전환 수술을 결심하게 된다. 엘베는 더 이상 결혼 생활이 힘들다는 것을 깨닫고 그들의 결혼 생활은 위기를 맞이하게 되지만 게르다는 남편의 결정을 존중한다. 또한 엘베의 어린 시절 친구 한스 악스질 가 등장하게 되면서 이들 부부와 복잡한 삼각 관계를 형성하게 된다. 7 8
### Question 3722:
Prompt: 유럽미래 협의회는 언제 만들어졌나요?

### Generated Completion:
1997년

### Original Completion:
2011년
### Question 3723:
Prompt: 김병관 성우가 애니메이션 라이온 킹에서 연기했던 배역의 이름은 무엇인가?

### Generated Completion:
이현수

### Original Completion:
스카
### Question 3724:
Prompt: 기원후 2세기부터 로마에서 일반인들이 사용한 구어는?

### Generated Completion:
구어

### Original Completion:
민중 라틴어
### Question 3725:
Prompt: 캐슬링의 의도가 있었다면 룩부터 만졌어도 캐슬링을 허용하는 협회는?

### Generated Completion:
협회

### Original Completion:
미국의 체스 협회
### Question 3726:
Prompt: 구결과 이두의 공통점은 무엇일까?

### Generated Completion:
저는 인공지능 어시스턴트이기 때문에 정확한 답변을 

### Original Completion:
김연아가 쇼트 프리 총점에 모두 세계 신기록을 세우며 우승 했던 2010년 밴쿠버 동계 올림픽을 계기
### Question 3756:
Prompt: 꽃다지의 생육특성은?

### Generated Completion:
봄부터 가을까지 일조시간도 높기 때문에 꽃다지를 키우는 것은 쉽지 않다. 그러나 대부분의 꽃다지는 꽃이 피기 전에 충분한 수면을 취하는 것이 좋다. 또한 햇볕이 잘 드는 날씨에는 꽃을 피우기 좋은 계절이기도 하다.

### Original Completion:
이른 봄 햇빛이 잘 드는 밭 논두렁 산기슭에서 노란색 꽃을 피운다. 높이는 10 20 센티미터 정도이다. 3 보통 가지가 많이 갈라지고 잎과 줄기에는 별 모양의 짧은 털이 빽빽이 나 있다. 뿌리에서 나오는 잎은 수가 많고 방석처럼 퍼진다. 6 뿌리에서 나는 잎은 주걱 모양이고 길이 2 4센티미터 너비 0.8 1.5센티미터 정도이다. 7 줄기에 있는 잎은 어긋나고 잎자루가 없다. 6 좁은 난형이거나 긴 타원형이고 길이 1 3센티미터 너비 0.5 1.5센티미터쯤 된다. 7 꽃은 4 6월에 줄기 끝에서 노란색으로 모여 핀다. 꽃잎은 네 장이며 수술은 여섯 개이다. 6 꽃자루는 옆으로 퍼지고 암술대는 매우 짧아 없는 것처럼 보인다. 7 꽃다지와 냉이는 사는 곳이 겹치는데 잎 모양으로 쉽게 구분할 수 있지만 꽃은 색만 다를 뿐 흡사하다. 냉이 꽃은 희고 꽃다지 꽃은 노랗다. 3 열매는 긴 타원형이고 전체에 털이 나 있으며 결실기는 7 8월로 6 꼬투리가 열리는 각과이다. 7 꼬투리에는 두 개의 방에 여남은 개의 씨앗이 들어있다. 3
### Question 3757:
Prompt: 2012년 승리가 첫 번째 솔로 팬 미팅을 가진 도시는?

### Generated Completion:
뉴욕

### Original Completion:
도쿄
### Question 3758:
Prompt: 발화가 시작되고 숭례문의 2층 문루 90 1층 문루 10 가 소실되

7월 29일 오후 0시
### Question 3997:
Prompt: 타블로의 노래 airbag에서 피쳐링을 한 가수의 이름은?

### Generated Completion:
제이크

### Original Completion:
나얼
### Question 3998:
Prompt: 암트랙을 이용하는 사람은 연간 약 몇만명인가?

### Generated Completion:
약 100만명

### Original Completion:
800만명
### Question 3999:
Prompt: 소크라테스는 나쁜 행동이나 악을 무엇의 결과라고 말했을까?

### Generated Completion:
악

### Original Completion:
무지
### Question 4000:
Prompt: 미국의 시장규모는 세계 몇위인가요?

### Generated Completion:
2위

### Original Completion:
2위
### Question 4001:
Prompt: 청대에 16년간 만든 곳이며 언덕지형을 가지고 강한 포대를 가진 난공불락의 요새로 알려진 해군 주둔지는 어디인가?

### Generated Completion:
부산

### Original Completion:
여순 해군 기지
### Question 4002:
Prompt: 이해랑은 일본의 어느 대학교 출신입니까?

### Generated Completion:
도쿄 대학

### Original Completion:
니혼 대학교
### Question 4003:
Prompt: 글라우버가 태어난 연도는 언제인가?

### Generated Completion:
1995년

### Original Completion:
1925년
### Question 4004:
Prompt: 원술이 황제가 되자 양봉은 어떤일을 겪게 되었을까요?

### Generated Completion:
양봉이란 왕이 된 후 자신의 뜻을 펼칠 수 있는 기회가 주어졌다

### Original Compl

In [59]:
# 저장 경로 설정
output_path = "/aiffel/aiffel/GD9/dataset/prompt_least_to_most_model_competition.jsonl"

# 비교 결과 저장
with open(output_path, "w", encoding="utf-8") as f:
    for i in range(len(test_data)):
        comparison_entry = {
            "question_number": i + 1,
            "prompt": test_data[i]['prompt'],
            "generated_completion": generated_completions[i],
            "original_completion": test_data[i]['completion']
        }
        # JSON Lines 형식으로 저장
        f.write(json.dumps(comparison_entry, ensure_ascii=False) + "\n")

print(f"비교 결과가 {output_path}에 저장되었습니다.")

비교 결과가 /aiffel/aiffel/GD9/dataset/prompt_least_to_most_model_competition.jsonl에 저장되었습니다.
